In [1]:
from modelloFaccia import compute_ids

In [2]:
from modelloFaccia import load,compute_ids
model = load()

Loading weights from /Users/paolocursi/Desktop/multimodal/tinyvit_face_best.pth
Weights loaded successfully


In [3]:
K = 10

In [ ]:
gallery = compute_ids(K)
https://github.com/robmsmt/ASR-Audio-Data-Links?tab=readme-ov-file#:~:text=Voxforge-,Voxforge%20English,-Read 

In [3]:
import requests
import zipfile
import os
from pathlib import Path
import tarfile

# Create directory for dataset
dataset_dir = Path("voxforge_dataset")
dataset_dir.mkdir(exist_ok=True)

# VoxForge English dataset URL
url = "http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/voxforge_corpus_16khz.tgz"

print("Downloading VoxForge English dataset...")
response = requests.get(url, stream=True)

if response.status_code == 200:
    # Save the file
    file_path = dataset_dir / "voxforge_corpus_16khz.tgz"
    with open(file_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    
    print(f"Dataset downloaded to {file_path}")
    
    # Extract the archive
    print("Extracting dataset...")
    with tarfile.open(file_path, 'r:gz') as tar:
        tar.extractall(dataset_dir)
    
    print("Dataset extracted successfully!")
else:
    print(f"Failed to download dataset. Status code: {response.status_code}")

Failed to download dataset. Status code: 404


In [ ]:
import whisper
import librosa
import numpy as np
import torch

# Load a lightweight Whisper model for speech embeddings
speech_model = whisper.load_model("tiny")
import os, certifi

# Point SSL to certifi’s CA bundle
os.environ["SSL_CERT_FILE"] = certifi.where()

def extract_speech_embedding(audio_path, duration=5):
    """
    Extract speech embedding from audio file
    Args:
        audio_path: Path to audio file
        duration: Duration in seconds to extract (default 5s)
    Returns:
        Speech embedding tensor
    """
    # Load audio file
    audio, sr = librosa.load(audio_path, sr=16000, duration=duration)
    
    # Ensure audio is the right length
    if len(audio) < 16000 * duration:
        # Pad with zeros if too short
        audio = np.pad(audio, (0, 16000 * duration - len(audio)))
    else:
        # Truncate if too long
        audio = audio[:16000 * duration]
    
    # Convert to tensor
    audio_tensor = torch.from_numpy(audio).float()
    
    # Extract features using Whisper encoder
    with torch.no_grad():
        # Whisper expects log-mel spectrogram
        mel = whisper.log_mel_spectrogram(audio_tensor)
        embedding = speech_model.encoder(mel.unsqueeze(0))
        
    return embedding.mean(dim=1).squeeze(0)  # Average over time dimension

print("Speech model loaded successfully!")

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1010)>

In [ ]:
import cv2
from collections import deque
import time
from PIL import Image
import torch
import torch.nn.functional as F

# Parameters
k = K  # Queue size for embeddings
alpha = 0.7  # Similarity threshold
frame_interval = 5  # Process embeddings every 10 frames
unknown_timeout = 5.0  # Seconds before adding new user

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam")
    exit()

# Initialize face detector (lightweight)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize embedding queue
embedding_queue = deque(maxlen=k)

# Frame counter
frame_count = 0

# Variables to store recognition results
current_match = None
current_similarity = 0.0

# Variables for new user registration
unknown_start_time = None
registration_mode = False
registration_embeddings = deque(maxlen=k)
registration_frame_count = 0
new_user_counter = 1

print("Starting face recognition... Press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame")
        break
    
    frame_count += 1
    current_time = time.time()
    
    # Convert to grayscale for face detection (every frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces (every frame)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    if len(faces) > 0:
        # Take the largest face (assuming closest to camera)
        largest_face = max(faces, key=lambda f: f[2] * f[3])
        x, y, w, h = largest_face
        
        # Process embeddings only every 10th frame
        if frame_count % frame_interval == 0:
            # Extract face region with some padding
            padding = 20
            x1 = max(0, x - padding)
            y1 = max(0, y - padding)
            x2 = min(frame.shape[1], x + w + padding)
            y2 = min(frame.shape[0], y + h + padding)
            
            face_img = frame[y1:y2, x1:x2]
            
            if face_img.size > 0:
                # Convert to PIL Image and apply transforms
                face_pil = Image.fromarray(cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB))
                face_tensor = val_tfms(face_pil).unsqueeze(0).to(device)
                
                # Get embedding
                with torch.no_grad():
                    embedding = model(face_tensor, return_embedding=True).squeeze(0)
                
                # Add to queue (automatically removes oldest if queue is full)
                embedding_queue.append(embedding)
                
                # If in registration mode, also add to registration queue
                if registration_mode:
                    registration_embeddings.append(embedding)
                    registration_frame_count += 1
                    
                    # If we have enough embeddings for registration
                    if len(registration_embeddings) >= k:
                        # Compute average embedding for new user
                        avg_registration_embedding = torch.stack(list(registration_embeddings)).mean(dim=0)
                        
                        # Add new user to gallery
                        new_user_name = f"NEW_USER_{new_user_counter}"
                        gallery[new_user_name] = avg_registration_embedding
                        new_user_counter += 1
                        
                        print(f"Added new user: {new_user_name}")
                        
                        # Reset registration mode
                        registration_mode = False
                        registration_embeddings.clear()
                        registration_frame_count = 0
                        unknown_start_time = None
                
                # If we have enough embeddings, compute average and compare
                if len(embedding_queue) >= k:
                    # Compute average embedding
                    avg_embedding = torch.stack(list(embedding_queue)).mean(dim=0)
                    
                    # Compare with gallery
                    best_match = None
                    best_similarity = 0.0
                    
                    for identity_name, embeddings in gallery.items():
                        # Compare with both tensors in gallery
                        sim = F.cosine_similarity(avg_embedding.unsqueeze(0), embeddings.unsqueeze(0)).item()

                        # Take the maximum similarity
                        if sim > best_similarity:
                            best_similarity = sim
                            best_match = identity_name
                    
                    # Update current recognition results
                    current_match = best_match
                    current_similarity = best_similarity
                    
                    if best_similarity > alpha:
                        print(f"Recognized: {best_match} with similarity {best_similarity:.3f}")
                        # Reset unknown timer since we found a match
                        unknown_start_time = None
                        registration_mode = False
                    else:
                        # Person is unknown
                        if unknown_start_time is None:
                            unknown_start_time = current_time
                        
                        # Check if unknown for long enough and not already in registration mode
                        elapsed = current_time - unknown_start_time
                        if elapsed >= unknown_timeout and not registration_mode:
                            print(f"Unknown person detected for {elapsed:.1f}s. Starting registration...")
                            registration_mode = True
                            registration_embeddings.clear()
                            registration_frame_count = 0
                
                # Clean up
                del face_tensor, embedding
                torch.cuda.empty_cache()
        
        # Draw face rectangle (every frame)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Display current recognition result (every frame)
        if current_match is not None and current_similarity > alpha:
            text = f"{current_match}: {current_similarity:.3f}"
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                      0.6, (0, 255, 0), 2)
        elif current_match is not None:
            # Show registration status
            if registration_mode:
                elapsed = current_time - unknown_start_time if unknown_start_time else 0
                text = f"Registering... ({len(registration_embeddings)}/{k})"
                cv2.putText(frame, text, (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 
                          0.6, (0, 255, 255), 2)
            
            text = f"Unknown: {current_similarity:.3f}"
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                      0.6, (0, 0, 255), 2)
    else:
        # No face detected, reset unknown timer
        unknown_start_time = None
        registration_mode = False
    
    # Display the frame
    cv2.imshow('Face Recognition', frame)
    
    # Break on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()
print("Face recognition stopped")

2025-06-10 12:58:47.410 Python[10871:558587] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Starting face recognition... Press 'q' to quit
Unknown person detected for 5.1s. Starting registration...
Added new user: NEW_USER_1
Recognized: NEW_USER_1 with similarity 1.000
Recognized: NEW_USER_1 with similarity 0.998
Recognized: NEW_USER_1 with similarity 0.992
Recognized: NEW_USER_1 with similarity 0.986
Recognized: NEW_USER_1 with similarity 0.976
Recognized: NEW_USER_1 with similarity 0.972
Recognized: NEW_USER_1 with similarity 0.972
Recognized: NEW_USER_1 with similarity 0.969
Recognized: NEW_USER_1 with similarity 0.966
Recognized: NEW_USER_1 with similarity 0.970
Recognized: NEW_USER_1 with similarity 0.967
Recognized: NEW_USER_1 with similarity 0.952
Recognized: NEW_USER_1 with similarity 0.941
Recognized: NEW_USER_1 with similarity 0.919
Recognized: NEW_USER_1 with similarity 0.882
Recognized: NEW_USER_1 with similarity 0.846
Recognized: NEW_USER_1 with similarity 0.780
Unknown person detected for 5.3s. Starting registration...
Unknown person detected for 5.3s. Starting 

: 

In [1]:
gallery.keys()

NameError: name 'gallery' is not defined